### 1. 介绍

- **描述**：Isomap（等距映射）的算法用于计算一组高维数据点的拟等距（基于欧氏距离）、低维嵌入。

- **步骤**：

  1. **近邻搜索**：对于每个数据点，找到其最近的`k`个邻居。
  2. **连接性**：构建一个连接图，其中每个数据点与其`k`个最近邻居相连。
  3. **最短路径**：计算每对数据点之间的最短路径距离，例如Dijkstra算法。
  4. **低维嵌入**：使用多维缩放（MDS）将高维数据点映射到低维空间。

- **缺点**：

  1. 选择“刚刚好”的邻域需要对高维数据流形的全局几何信息有先验了解（参数敏感）
  2. 在上拓扑不稳定，对分布结构的要求较高(例如非闭合流形的两端距离若接近，则有可能导致测地距离计算错误)。
  3. 对噪声点敏感，容易出现短路错误（Short-circuit error）
     - 短路错误：指计算数据点之间的距离时，由于选择的邻域过大或数据噪声的影响，导致引入了不应存在的连接。 
     - 例子：假设有一个数据点A和一个数据点B，它们在高维空间中有很远的距离，但由于某些噪声或邻域选择不当，它们在构建的邻域图中被错误地连接起来。这种错误连接就会影响到后续的测地距离计算，最终导致低维嵌入的错误。

- **优点**：

  1. 保留了数据点之间的全局几何信息，适用于非线性流形的降维。
  2. 通过最短路径距离，避免了高维空间中的直线距离计算，更适用于非线性流形。

### 2. 流程


Isomap（Isometric Mapping）是一种用于降维的非线性方法，它在保持高维数据点之间的测地距离的基础上，计算出其低维嵌入。Isomap算法的步骤可以总结如下：
1. **构建距离矩阵：**

   - 计算数据集中所有点之间的欧氏距离，得到距离矩阵 $D$。
   - 对于每个数据点，找到其 $k$ 个最近邻或根据距离阈值找到邻居点。这一步可以使用 $k$-最近邻算法（k-NN）或 $\epsilon$-邻域方法。
2. **构建邻接图：**

   - 基于最近邻关系构建图 $G$，将每个数据点作为图中的一个节点，如果两个点互为最近邻，则在它们之间连一条边。边的权重为两点之间的欧氏距离。
  
3. **计算测地距离矩阵：**
   - 使用最短路径算法（例如Dijkstra或Floyd-Warshall算法），在邻接图 $G$ 上计算任意两点之间的最短路径距离，得到测地距离矩阵 $D_G$。
4. **多维尺度分析（MDS）：**
   - 对测地距离矩阵 $D_G$ 进行中心化处理，将其转换为内积矩阵。
   - 使用特征值分解技术，对内积矩阵进行特征值分解，选择前 $d$ 个最大的特征值及其对应的特征向量，得到低维嵌入。

具体步骤如下：
#### A. 构建距离矩阵
对于每一对数据点 $i$ 和 $j$，计算它们之间的欧氏距离：
$$ d_{ij} = \| x_i - x_j \| $$
#### B. 构建邻接图
构建一个图 $G$，其中每个数据点作为一个节点，边的权重为欧氏距离。仅保留每个节点的 $k$ 个最近邻的边或距离小于 $\epsilon$ 的边。
#### C. 计算测地距离矩阵
使用最短路径算法（例如Dijkstra或Floyd-Warshall算法）计算图 $G$ 上的任意两点 $i$ 和 $j$ 之间的测地距离 $d_G(i,j)$。
#### D. 多维尺度分析（MDS）

1. **中心化处理：**
   $$ B = -\frac{1}{2} H D_G^2 H $$
   其中 $ H = I - \frac{1}{n} \mathbf{1} \mathbf{1}^T $，$I$ 是单位矩阵，$\mathbf{1}$ 是全1向量。
2. **特征值分解：**
   对矩阵 $B$ 进行特征值分解：
   $$ B = V \Lambda V^T $$
   其中 $\Lambda$ 是特征值对角矩阵，$V$ 是特征向量矩阵。
3. **选择前 $d$ 个最大的特征值及其对应的特征向量：**
   $$ Y = V_d \Lambda_d^{1/2} $$
   其中 $V_d$ 包含前 $d$ 个最大的特征向量，$\Lambda_d$ 是对应的特征值对角矩阵。
最终得到的数据矩阵 $Y$ 就是高维数据在低维空间中的嵌入。

#### E. 总结
Isomap通过保持高维数据点之间的测地距离，实现了非线性降维，能更好地揭示高维数据中的底层结构。